## Doamore - a diamond ring shop   webscraping.

Doamore is a diamond ring shop where you can buy diamond rings. My project is to scrape the details of the diamonds.I want to extract the following details.

1. SHAPE     2. CARATS   3. COLOR   4. CLARITY   5. CUT  6. RATIO  7. TABLE  8. DEPTH  9. MEASUREMENTS 10. PRICE	11. ID
12. STOCK_NO	13. IMAGE_URL	14. VIDEO_URL	15. DETAIL_URL	16. CERTIFICATION	17. CERTIFICATE NO	18. FLOURESCENCE	

As a first step we are importing the required modules.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import re
import time

The next step in  web scraping is to deeply analysr the website and finding out  the details we want.

In our case, the base url is https://www.doamore.com/lab-created-diamonds.

But our details are in this url : https://www.doamore.com/lab-created-diamonds/?meta={%22carat%22:{%22min%22:0.24,%22max%22:6.91},%22ratio%22:{%22min%22:1,%22max%22:3.15}}&taxonomies={}&page=2&thumbnails=false&cert_igi=false&cert_gcal=false&cert_gia=false

There are 9525 diamond results which are listed over 477 pages. 

So we have to iterate over the 477 pages. Only the page number changes  in the url when we go to the next page.


The python code below provide  iteration over the pages.

In [3]:
base_url = "https://www.doamore.com/lab-created-diamonds/?meta={%22carat%22:{%22min%22:0.24,%22max%22:6.91},%22ratio%22:{%22min%22:1,%22max%22:3.15}}&taxonomies={}&page="
last_url = "&thumbnails=false&cert_igi=false&cert_gcal=false&cert_gia=false" 
for i in range(1,3):
    topic_url =base_url+str(i)+last_url
    print(topic_url)

https://www.doamore.com/lab-created-diamonds/?meta={%22carat%22:{%22min%22:0.24,%22max%22:6.91},%22ratio%22:{%22min%22:1,%22max%22:3.15}}&taxonomies={}&page=1&thumbnails=false&cert_igi=false&cert_gcal=false&cert_gia=false
https://www.doamore.com/lab-created-diamonds/?meta={%22carat%22:{%22min%22:0.24,%22max%22:6.91},%22ratio%22:{%22min%22:1,%22max%22:3.15}}&taxonomies={}&page=2&thumbnails=false&cert_igi=false&cert_gcal=false&cert_gia=false


code above is an exapmle with only 2 pages. This will work through out the 477 pages. The range should be (1,478)

In [4]:
base_url = "https://www.doamore.com/lab-created-diamonds/?meta={%22carat%22:{%22min%22:0.24,%22max%22:6.91},%22ratio%22:{%22min%22:1,%22max%22:3.15}}&taxonomies={}&page="
last_url = "&thumbnails=false&cert_igi=false&cert_gcal=false&cert_gia=false" 
for i in range(1,478):
    topic_url =base_url+str(i)+last_url

We want to extract the html document from each of the page  in the loop. Let us examine with one page.Then we will apply to the loop

The next step is to extract the html document

In [6]:
headers = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}
response = requests.get(topic_url, headers = headers, timeout = 10)
page_contents = response.text

then  parsing the deocument into a variable doc.

In [7]:
doc = BeautifulSoup(page_contents,'html.parser')

Our required data is in table format. So we are going to find the table in the doc.

In [8]:
table = doc.find_all('table')

In [12]:
table

[<table>
 <thead>
 <tr>
 <td>Shape</td>
 <td>Carats</td>
 <td>Cut</td>
 <td>Color</td>
 <td>Clarity</td>
 <td class="extra">L/W Ratio</td>
 <td class="extra">Measurements</td>
 <td>Price</td>
 </tr>
 </thead>
 <tbody id="diamonds">
 <tr class="diamond view" data-id="3863642">
 <td>Oval</td>
 <td>5.00</td>
 <td>Very Good</td>
 <td>D</td>
 <td>VVS2</td>
 <td class="extra">1.38</td>
 <td class="extra">13.44 x 9.73 x 5.91mm</td>
 <td>$66,020</td>
 </tr>
 <tr class="diamond view" data-id="4113865">
 <td>Oval</td>
 <td>5.03</td>
 <td>Excellent</td>
 <td>D</td>
 <td>VVS2</td>
 <td class="extra">1.42</td>
 <td class="extra">13.90 x 9.76 x 5.80mm</td>
 <td>$66,420</td>
 </tr>
 <tr class="diamond view" data-id="3352795">
 <td>Round</td>
 <td>5.76</td>
 <td>Signature Ideal</td>
 <td>F</td>
 <td>VS1</td>
 <td class="extra">1.00</td>
 <td class="extra">11.46 x 11.53 x 7.11mm</td>
 <td>$66,588</td>
 </tr>
 <tr class="diamond view" data-id="3820592">
 <td>Cushion</td>
 <td>5.33</td>
 <td>Very Good</t

This is an ajax website.So inorder to extract the table details, we would get the json document using a post request.

In [14]:
id_list = []
for tag in table[0].find_all(class_='diamond view'):
        id_single =tag.get('data-id')
        
        id_list.append(id_single)
    
        header2 = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:91.0) Gecko/20100101 Firefox/91.0',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Accept-Language': 'en-CA,en-US;q=0.7,en;q=0.3',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'X-Requested-With': 'XMLHttpRequest',
            'Origin': 'https://www.doamore.com',
            'Alt-Used': 'www.doamore.com',
            'Connection': 'keep-alive',
            'Referer': 'https://www.doamore.com/lab-created-diamonds/',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin',
                        }
        data2 = {'action': 'diamond_view','id':id_single}
        
        url = "https://www.doamore.com/wp-admin/admin-ajax.php"
        response2 = requests.post(url, headers = header2, data = data2,timeout = 10)

From the response object we are extracting the json data using beautifulsoup.

In [15]:
        data_decoded = response2.text.encode().decode('utf-8-sig')
        data_json = json.loads(data_decoded)
        new_data = data_json['data']
        soup = BeautifulSoup(data_json['data'],'html.parser')

Getting  the details out of the json data

In [16]:
 #getting the stock no and details url
        detail_url = soup.find_all('a')
        details_url = detail_url[0]['href']
        detail_url_list.append(details_url)
        stock_no = details_url.split("/")[-2:-1]
        stock_no_list.append(stock_no[0])
        
        #get the certificate number and certificate url
        url_cert = 'https://www.doamore.com/lab-created-diamonds/'+str(stock_no[0])
        response = requests.get(url_cert, headers=headers,timeout= 10)
        cert_soup = BeautifulSoup(response.text,'html.parser')

        links = cert_soup.find_all('a',text=["IGI","GCAL","GSI","GIA","Diamond Foundry","GIA-N"])
        cert_url = links[0]['href']
        print(cert_url)
        cert_url_list.append(cert_url)
        cert_no = cert_url.split("=")[-1:]
        if(len(cert_no[0])==0):
            certificate_number = nil
        elif(len(cert_no[0])<=10):
            certificate_number= "LG"+str(cert_no[0])
        else:
            cert_no = cert_url.split("/")[-1:]
            certificate_number = cert_no[0].split('.')[0] 
        
        cert_num_list.append(certificate_number)
        
        #getting flourescence value
        frsnce_data =cert_soup.find_all('div',{'class':'dimensions'})
        flourescence = frsnce_data[0].find_all("li")
        flr_value =flourescence[4].text.split(':')[1]
         flouresence_list.append(flr_value)
        
        #getting video url
        video = cert_soup.find_all('iframe')
        if (len(video)<=1):
            video_url = nil
        else:
            video_url = video[1]['src']
        video_url_list.append(video_url)
        
        
        
    
        #getting image url
        image_url = soup.find_all('img')
        if (len(image_url)==0):
            img_src = nil
        else:
            img_src = image_url[0]['src']
        image_url_list.append(img_src)
        

        #getting the price details
        details = soup.find_all('p')
        price = details[0].text
        price_list.append(price)
        
        body = []
        rows.append(body)

IndentationError: unexpected indent (<ipython-input-16-7a9c8c31c032>, line 2)